In [ ]:
import sys
!{sys.executable} -m pip install sockets
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install igraph
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pyvis

In [1]:
import time

import numpy  as np
import igraph as ig
import matplotlib.pyplot as plt

plt.rc("text", usetex=True)

%run ../SocketServer.ipynb
%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./customizedLBP.ipynb

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [2]:
def factorLoader(factor_input, dilimiter):
    tokens = factor_input.split(dilimiter)
    for idx in range(0, len(tokens), 4):
        yield tokens[idx], tokens[idx+1], tokens[idx+2], tokens[idx+3]

In [3]:
def printNodeCount(graph):
    factor_node_count = 0
    var_node_count = 0
    
    for i in range(graph.vcount()):
        if graph.vs[i]['is_factor']:
            factor_node_count += 1
        else:
            var_node_count += 1
            
    print("var node count:", var_node_count, "factor node count:", factor_node_count, "total:", factor_node_count + var_node_count)
    print("edges count:", graph.ecount())

In [4]:
def checkDuplicateVar(str_):
    str_tokens = [i.split('(') for i in str_.split(')') if i != '']
    for token in str_tokens:
        vars = token[1].split(',')
        if len(vars) != len(set(vars)):
            print("contain duplicate variables: ", token[0])
            print(vars)
            raise ValueError('Duplicated variables: ' + token[0])

In [5]:
class BeliefPropagationServer(SocketServer):
    
    def __init__(self, host, port):
        super().__init__(host, port)
        self.MUL_SIGN = "*"
        self.MAX_ITR = 10
        
    def func(self):
        while True:
            print("-"*20)
            graph_input = self.recvMsg()
            factor_input = self.recvMsg()
            
            if not graph_input or not factor_input:
                response = ""
                self.sendMsg(response)
                print("receive empty message")
                break
            
            checkDuplicateVar(graph_input)
            if graph_input == self.END and factor_input == self.END:
                response = self.END
                self.sendMsg(response)
                self._stillWorking = False
                print("receive end message")
                break
                
            fg = string2factor_graph(graph_input)
            predIDs_all = set()
            for order, constraintID, predIDs_str, probs_str in factorLoader(factor_input, self.DILIMITER_2):
                try:
                    predIDs = predIDs_str.split(self.DILIMITER_1)

#                     print(predIDs)
                    predIDs_all.update(predIDs)
                    predCount = len(predIDs)

                    shape = [2 for _ in range(predCount)]
                    shape = tuple(shape)

                    probs_tokens = probs_str.split(self.DILIMITER_1)

                    probs = []
                    for probs_token in probs_tokens:
                        probs_str, count = probs_token.split(self.MUL_SIGN)
                        count = int(count)
                        prob = float(probs_str)
                        for _ in range(count):
                            probs.append(prob)

                    probs = np.array(probs)
                    probs = probs.reshape(shape)

                    fg.change_factor_distribution(constraintID, factor(predIDs,  probs))
                except Exception as e:
                    print(e)
                    print("constraintID:", constraintID)
                    print("predIDs_str:", predIDs_str)
                    print("probs_str:", probs_str)
                    return

            printNodeCount(fg.get_graph())

            lbp = myLBP(fg)
            start = time.time()
            margProb = lbp.belief(predIDs_all, self.MAX_ITR)
            end = time.time()

            print("time needed: ", end - start)
            plot_factor_graph(fg)
            output_str = ""
            for predID, prob in margProb.items():
#                         print(predID, prob)
                output_str += predID + self.DILIMITER_1 + str(prob)
                output_str += self.DILIMITER_2

            response = output_str[:-1]
            self.sendMsg(response)



In [6]:
# Server setting
HOST = "127.0.0.1"
PORT = 8080
STATEMENT_ID_PREFIX = "S_"

In [7]:
server = BeliefPropagationServer(HOST, PORT)

In [8]:
server.start()

Connected by ('127.0.0.1', 55065)
--------------------
var node count: 37 factor node count: 97 total: 134
edges count: 491
time needed:  19.62214159965515
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
response: b'org/apache/commons/math/analysis/ComposableFunction$10{117@117}d-4,0.42267761158076045&vir_org.apache.commons.math.analysis.ComposableFunction$18#value%D^D:2188,0.2437509371565431&vir_org.apache.commons.math.analysis.ComposableFunction#of%Lorg/apache/commons/math/analysis/UnivariateRealFunction;^Lorg/apache/commons/math/analysis/ComposableFunction;:2170,0.36532555408973977&org/apache/commons/math/analysis/ComposableFunction$28{283@283}this$0-3,0.22347430396714943&vir_org.apache.commons.math.analysis.ComposableFunction$5#value%D^D:2190,0.34435188930340055&org/apache/commons/math/analysis/ComposableFunctionTest{61@103}atan-1,0.47602247478200915&org.apache.commons.math.analysis.ComposableFunction.NEGATE,0.47925771294928027&org/apache/commons/

time needed:  0.1651160717010498
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
response: b'S_2173,0.7750913219820504&org/apache/commons/math/analysis/ComposableFunction$28{-1@-1}org.apache.commons.math.analysis.ComposableFunction$28.<init>.283:6-3,0.21979339984523422&org/apache/commons/math/analysis/ComposableFunction$28{283@283}this$0-3,0.21979339984523422&310350177.val$f,0.5412464206937694&310350177.this$0,0.5412464206937694'
--------------------
response: b''
receive empty message
Connected by ('127.0.0.1', 55077)
--------------------
var node count: 3 factor node count: 5 total: 8
edges count: 11
time needed:  0.0895528793334961
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
response: b'S_2176,0.6450163123142573&vir_org.apache.commons.math.analysis.ComposableFunction#of%Lorg/apache/commons/math/analysis/UnivariateRealFunction;^Lorg/apache/commons/math/analysis/ComposableFunction;:2176,0.48142593936988826&org/a

response: b'org/apache/commons/math/analysis/ComposableFunction$10{117@117}d-4,0.4183097920393999&vir_org.apache.commons.math.analysis.ComposableFunction$10#value%D^D:2192,0.4337883606542976&S_2192,0.515691340510923&CR_2184,0.1111445398754694&vir_org.apache.commons.math.analysis.ComposableFunction$28#value%D^D:2191,0.4183097920393999'
--------------------
response: b''
receive empty message
Connected by ('127.0.0.1', 55089)
--------------------
var node count: 7 factor node count: 9 total: 16
edges count: 27
time needed:  0.2539989948272705
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
response: b'vir_org.apache.commons.math.analysis.ComposableFunction$18#value%D^D:2188,0.24032946611018613&vir_org.apache.commons.math.analysis.ComposableFunction$28#value%D^D:2193,0.43755319907588713&S_2193,0.5286920535107362&vir_org.apache.commons.math.analysis.ComposableFunction$5#value%D^D:2190,0.3402837541699087&org/apache/commons/math/analysis/ComposableFunction$

KeyboardInterrupt: 